# VizProo: Visualizaciones Interactivas y Dashboards Inteligentes con Python

Este notebook demuestra el uso de **VizProo**, una librería diseñada para crear visualizaciones interactivas y dashboards avanzados **usando muy pocas líneas de código**, construida sobre **D3.js** y conectada de forma nativa al kernel de Python.

El objetivo es mostrar:

- Qué problema soluciona VizProo  
- Su arquitectura (Kernel ↔ UI)  
- Sus principales componentes visuales  
- Un caso de uso real con datos multidimensionales (Wine dataset)  
- Cómo construir dashboards profesionales e interactivos con mínimo código  

## ¿Qué problema soluciona VizProo?

VizProo nace para resolver un problema común en ciencia de datos:

> **Construir visualizaciones avanzadas y dashboards interactivos requiere muchas líneas de código, librerías web y gran complejidad técnica.**

VizProo elimina esa complejidad permitiendo:

- Crear visualizaciones profesionales con apenas 1–2 líneas  
- Conectar gráficos entre sí sin callbacks largos  
- Construir dashboards completos usando `MatrixCreator`  
- Realizar análisis multidimensional interactivo (RadViz, Star Coordinates, ScatterPlot, BarPlot)  
- Integrar cálculos del kernel como PCA o t-SNE con gráficos dinámicos  

---

## Arquitectura de VizProo (Kernel ↔ UI)

VizProo funciona mediante un modelo híbrido:
Python Kernel <-> VizProo UI (JavaScript + D3.js)


- **Python**: Ejecuta cálculos, cargas de datos, filtros, modelos (ej. t-SNE).  
- **D3.js + JS**: Renderiza gráficos, maneja selección de puntos e interactividad.  

Cada vez que seleccionas puntos en un gráfico:

1. La selección viaja al **kernel**.  
2. El kernel filtra datos o ejecuta cálculos.  
3. El resultado regresa a la **UI** y actualiza el resto de gráficos.  

Así es posible crear dashboards en minutos, sin HTML, CSS ni JavaScript.

---

## Características principales

- Visualizaciones avanzadas (RadViz, StarCoordinates, BarPlot, ScatterPlot)  
- Dashboards mediante `MatrixCreator`(nos ayuda a ordenar las visualizaciones)  
- Comunicación Kernel ↔ UI  
- Selecciones que actualizan otros gráficos  
- Código limpio, corto y escalable  

---

## Caso de uso real

En este notebook construiremos un dashboard interactivo para explorar el **Wine dataset**:

- RadViz para visualizar variables multidimensionales  
- Star Coordinates para interpretación geométrica  
- BarPlot para diferencias por clase  
- t-SNE para reducción de dimensionalidad  
- Dashboard completo con 1 sola matriz  



## Ordenar el layout
Layout de visualizaciones crear las secciones donde ira cada visualización. Código simple y practico solo necesita 2 apramtros que es row y columns.

In [40]:
from vizproo import MatrixCreator

m = MatrixCreator(
    rows=4,
    columns=6
)
m


MatrixCreator(columns='6', rows='4')

## Import dataset - Wine

Se uso el dataset de vinos de la libreria Sklearn; la selección se basa en la diversidad del dataset

In [41]:
from vizproo import RadViz, StarCoordinates, ScatterPlot, BarPlot, MatrixLayout, Button
from sklearn.manifold import TSNE
from sklearn.datasets import load_wine
import pandas as pd

# Cargar dataset Wine 
wine = load_wine()
df_wine = pd.DataFrame(wine.data, columns=wine.feature_names)
df_wine['target'] = wine.target.astype(str)

#Dimensiones numéricas a analizar 
dimensions = [
    'alcohol',
    'malic_acid',
    'ash',
    'flavanoids',
    'color_intensity',
    'proline'
]

df_wine.head()


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


## Creación de visualizaciones

In [42]:

# Visualizaciones principales
plot1 = RadViz(data=df_wine, dimensions=dimensions, hue='target')
plot2 = StarCoordinates(data=df_wine, dimensions=dimensions, hue='target')

# Botón para ejecutar t-SNE
button = Button(description="Calcular t-SNE")

# Plot vacío para t-SNE
scatterplot = ScatterPlot(data=pd.DataFrame(), x='dim1', y='dim2', hue='color')

# BarPlot dinámico
barplot = BarPlot(data=df_wine, x='target', y='alcohol')



def on_plot1_selected(change):
    df = plot1.selectedValues
    if not df.empty:
        plot2.data = df
        barplot.data = df
    else:
        plot2.data = df_wine
        barplot.data = df_wine

plot1.on_select_values(on_plot1_selected)



def calculate_TSNE(_):
    df = plot1.selectedValues if not plot1.selectedValues.empty else df_wine
    
    X = df[dimensions].values
    y = df['target'].values

    n = len(df)
    if n < 3:
        return

    perplexity = min(30, max(5, n - 1))
    
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    X_tsne = tsne.fit_transform(X)

    tsne_df = pd.DataFrame({
        'dim1': X_tsne[:, 0],
        'dim2': X_tsne[:, 1],
        'color': y
    })

    scatterplot.data = tsne_df

button.on_click(calculate_TSNE)


## DASHBOARD - FINAL

In [44]:
matrix = m.data
matrixLayout = MatrixLayout(matrix)

# Organización del dashboard
matrixLayout.add(plot1, 1)       # RadViz
matrixLayout.add(plot2, 2)       # Star Coordinates
matrixLayout.add(button, 3)      # Botón
matrixLayout.add(barplot, 4)     # BarPlot
matrixLayout.add(scatterplot, 5) # ScatterPlot t-SNE

matrixLayout


RadViz(dataRecords=[{'alcohol': 14.23, 'malic_acid': 1.71, 'ash': 2.43, 'alcalinity_of_ash': 15.6, 'magnesium'…

StarCoordinates(dataRecords=[{'alcohol': 13.3, 'malic_acid': 1.72, 'ash': 2.14, 'alcalinity_of_ash': 17.0, 'ma…

Button(description='Calcular t-SNE', elementId='ipcDXuDjbg')

BarPlot(dataRecords=[{'alcohol': 13.3, 'malic_acid': 1.72, 'ash': 2.14, 'alcalinity_of_ash': 17.0, 'magnesium'…

ScatterPlot(elementId='oHHwpWxrjC', hue='color', x='dim1', y='dim2')

MatrixLayout(grid_areas=['KbiZSfqhFZ', 'DBiBCCdFee', 'ipcDXuDjbg', 'qAqgEjtfBw', 'oHHwpWxrjC'], grid_template_…

## Selección de valores por visaulización
Interactividad con las gráficas 

In [45]:
barplot.selectedValues

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.37,1.95,2.5,16.8,113,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480,0


In [46]:
plot1.selectedValues

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,0
1,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450,0
2,13.86,1.35,2.27,16.0,98,2.98,3.15,0.22,1.85,7.22,1.01,3.55,1045,0
3,14.10,2.16,2.30,18.0,105,2.95,3.32,0.22,2.38,5.75,1.25,3.17,1510,0
4,14.12,1.48,2.32,16.8,95,2.20,2.43,0.26,1.57,5.00,1.17,2.82,1280,0
5,13.75,1.73,2.41,16.0,89,2.60,2.76,0.29,1.81,5.60,1.15,2.90,1320,0
6,14.38,1.87,2.38,12.0,102,3.30,3.64,0.29,2.96,7.50,1.20,3.00,1547,0
7,14.19,1.59,2.48,16.5,108,3.30,3.93,0.32,1.86,8.70,1.23,2.82,1680,0
8,13.30,1.72,2.14,17.0,94,2.40,2.19,0.27,1.35,3.95,1.02,2.77,1285,0
9,13.58,1.66,2.36,19.1,106,2.86,3.19,0.22,1.95,6.90,1.09,2.88,1515,0


In [50]:
plot2.selectedValues

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,13.86,1.35,2.27,16.0,98,2.98,3.15,0.22,1.85,7.22,1.01,3.55,1045,0
1,14.19,1.59,2.48,16.5,108,3.30,3.93,0.32,1.86,8.70,1.23,2.82,1680,0
2,13.58,1.66,2.36,19.1,106,2.86,3.19,0.22,1.95,6.90,1.09,2.88,1515,0
3,13.94,1.73,2.27,17.4,108,2.88,3.54,0.32,2.08,8.90,1.12,3.10,1260,0
4,13.05,1.73,2.04,12.4,92,2.72,3.27,0.17,2.91,7.20,1.12,2.91,1150,0
5,12.21,1.19,1.75,16.8,151,1.85,1.28,0.14,2.50,2.85,1.28,3.07,718,1
6,12.58,1.29,2.10,20.0,103,1.48,0.58,0.53,1.40,7.60,0.58,1.55,640,2


In [49]:
scatterplot.selectedValues

,dim1,dim2,color
0,46.670399,18.896090,0
1,22.683695,-13.100032,0
2,65.383018,53.311195,0
3,13.116290,19.115423,0
4,61.874691,-16.980040,0
5,24.607294,51.104031,2


En conclusión, VizProo permite construir visualizaciones claras, estéticas y altamente interactivas utilizando muy pocas líneas de código. Gracias a su arquitectura basada en D3.js, la librería ofrece gráficos dinámicos que resultan intuitivos para el usuario final. Además, el sistema de matrices facilita la creación de dashboards completos sin necesidad de estructuras complejas, logrando un excelente equilibrio entre simplicidad, funcionalidad y diseño.